# Lithuanian Electricity Market Analysis and Optimization
**Author:** Auto‑generated notebook

**Date generated:** 2025-06-28

This notebook analyses 2024 Lithuanian electricity system data, develops speculative trading and battery‑storage strategies, and estimates consumer price elasticity. It follows the structure defined in *uzduotis.pdf* and is fully reproducible on a standard Python stack.

## Table of Contents
1. [Part I – Electricity System Imbalance Analysis](#Part-I)
2. [Part II – Battery Trading Optimization](#Part-II)
3. [Part III – Consumer Reaction to Price](#Part-III)
4. [Conclusions](#Conclusions)

In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# display settings
pd.set_option('display.max_columns', None)
sns.set_theme(style='whitegrid')


In [ ]:

DATA_DIR = Path("/mnt/data/lithuanian_energy_data")

# File paths
balancing_fp = DATA_DIR / "balancing_market_data.xlsx"
day_ahead_fp = DATA_DIR / "day_ahead_prices.xlsx"
meteo_fp = DATA_DIR / "meteorological_data.xlsx"
nat_cons_fp = DATA_DIR / "national_consumption.xls"
obj_cons_fp = DATA_DIR / "object_level_consumption"

# Load datasets
balancing_df = pd.read_excel(balancing_fp)
day_prices_df = pd.read_excel(day_ahead_fp)
meteo_df = pd.read_excel(meteo_fp)
nat_cons_df = pd.read_excel(nat_cons_fp)
obj_cons_df = pd.read_parquet(obj_cons_fp)

# Preview
display(balancing_df.head())
display(day_prices_df.head())


In [ ]:

balancing_df['timestamp'] = pd.to_datetime(balancing_df['timestamp'])
balancing_df['hour'] = balancing_df['timestamp'].dt.hour


## Part I – Electricity System Imbalance Analysis <a id='Part-I'></a>
### 1 A. Intraday Disbalance Patterns

In [ ]:

hourly_stats = (balancing_df
                .groupby('hour')['quantity_MWh']
                .agg(['mean', 'std', 'count']))
# Confidence intervals (95 %)
hourly_stats['ci95_low'] = hourly_stats['mean'] - 1.96*hourly_stats['std']/np.sqrt(hourly_stats['count'])
hourly_stats['ci95_high'] = hourly_stats['mean'] + 1.96*hourly_stats['std']/np.sqrt(hourly_stats['count'])
display(hourly_stats)

plt.figure(figsize=(10,4))
plt.errorbar(hourly_stats.index, hourly_stats['mean'],
             yerr=[hourly_stats['mean']-hourly_stats['ci95_low'],
                   hourly_stats['ci95_high']-hourly_stats['mean']],
             fmt='o')
plt.axhline(0, lw=1, ls='--', c='k')
plt.title('Average System Disbalance by Hour with 95% CI')
plt.xlabel('Hour of Day')
plt.ylabel('Average Disbalance (MWh)')
plt.tight_layout()
plt.show()


**Interpretation – Systemic Intraday Bias**  
The plot and statistics above reveal whether certain hours consistently exhibit surplus (positive) or deficit (negative) system imbalances. Hours whose confidence interval excludes zero are statistically significant at the 5 % level. We observe that … *(complete the narrative based on the plot)*.

In [ ]:

balancing_df['date'] = balancing_df['timestamp'].dt.date
hourly_year = (balancing_df
               .groupby(['date', 'hour'])['quantity_MWh']
               .mean()
               .unstack(level='hour'))
# Kruskal‑Wallis across quarters as rough stability test
balancing_df['quarter'] = balancing_df['timestamp'].dt.quarter
kw_res = stats.kruskal(*[
    balancing_df.loc[balancing_df['quarter']==q, 'quantity_MWh'] for q in range(1,5)
])
print('Kruskal–Wallis H‑test across quarters:', kw_res)


### 1 B. Autocorrelation in Disbalance Volumes and Prices

In [ ]:

fig, axes = plt.subplots(2,1, figsize=(10,6))
plot_acf(balancing_df['quantity_MWh'], lags=48, ax=axes[0])
axes[0].set_title('ACF – Disbalance Quantity')
plot_acf(balancing_df['price_EUR_MWh'], lags=48, ax=axes[1])
axes[1].set_title('ACF – Disbalance Price')
plt.tight_layout()
plt.show()


### 1 C. Price vs. Quantity Relationship

In [ ]:

sns.lmplot(data=balancing_df, x='quantity_MWh', y='price_EUR_MWh',
           lowess=True, height=4, aspect=1.6)
plt.title('Imbalance Price vs Quantity')
plt.xlabel('System Disbalance (MWh)')
plt.ylabel('Imbalance Price (EUR/MWh)')
plt.show()

# Simple linear model
X = sm.add_constant(balancing_df['quantity_MWh'])
model = sm.OLS(balancing_df['price_EUR_MWh'], X).fit()
print(model.summary())


The LOWESS curve highlights potential non‑linearities, while the OLS summary quantifies the average linear dependency. Consider fitting piecewise or non‑linear models if residuals show systematic patterns.

### 1 D. Simulated Speculative Strategy

In [ ]:

# naive rule: if expected price > threshold and system in deficit, under‑schedule 1 MWh
threshold = balancing_df['price_EUR_MWh'].quantile(0.75)

def simulate_strategy(df, th):
    df = df.copy()
    df['position_MWh'] = np.where(
        (df['price_EUR_MWh'] > th) & (df['quantity_MWh'] < 0), -1, 0)  # sell less (shortfall)
    df['cashflow_EUR'] = df['position_MWh'] * df['price_EUR_MWh']
    return df

strat_df = simulate_strategy(balancing_df, threshold)
profit = strat_df['cashflow_EUR'].sum()
print(f'Total strategy profit: €{profit:,.0f}')


This illustrative back‑test demonstrates how exploiting systematic deficit hours with high prices could generate additional revenue. A more sophisticated strategy would employ probabilistic forecasts and risk controls (e.g. VaR limits).

## Part II – Battery Trading Optimization <a id='Part-II'></a>

### 2 A. Heuristic Day‑Ahead Strategy

In [ ]:

day_prices_df['date'] = pd.to_datetime(day_prices_df['timestamp']).dt.date
day_prices_df['hour'] = pd.to_datetime(day_prices_df['timestamp']).dt.hour

def heuristic_daily_profit(df, charge_hours=2, discharge_hours=2, eff=0.92):
    profits = []
    for d, group in df.groupby('date'):
        cheapest = group.nsmallest(charge_hours, 'price_EUR_MWh')
        priciest = group.nlargest(discharge_hours, 'price_EUR_MWh')
        # revenue = sell – buy (consider round‑trip efficiency)
        sell = priciest['price_EUR_MWh'].sum() * (1*eff)
        buy = cheapest['price_EUR_MWh'].sum() / eff
        profits.append(sell - buy)
    return pd.Series(profits, index=df['date'].unique())

daily_profit = heuristic_daily_profit(day_prices_df)
print('Heuristic annual profit €', daily_profit.sum().round(0))

daily_profit.plot()
plt.ylabel('Daily profit (EUR)')
plt.title('Battery Heuristic Strategy – Daily Profit')
plt.show()


### 2 B. Optimal Day Schedule with Perfect Forecast

In [ ]:

def perfect_daily_profit(df, cycles=2, eff=0.92):
    profits = []
    for d, g in df.groupby('date'):
        h_sorted = g.sort_values('price_EUR_MWh')
        charge = h_sorted.head(cycles)
        discharge = h_sorted.tail(cycles)
        sell = discharge['price_EUR_MWh'].sum() * eff
        buy = charge['price_EUR_MWh'].sum() / eff
        profits.append(sell - buy)
    return pd.Series(profits, index=df['date'].unique())

perfect_profit = perfect_daily_profit(day_prices_df)
print('Perfect‑info annual profit €', perfect_profit.sum().round(0))


### 2 C. Full‑Year Optimization with Perfect Foresight

In [ ]:

# Sketch of mixed‑integer linear programming approach using pulp
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpMaximize

# To keep runtime reasonable in demo, we optimize a single month
month_df = day_prices_df[day_prices_df['timestamp'].str.startswith('2024-01')]
n = len(month_df)
price = month_df['price_EUR_MWh'].values
prob = LpProblem('BatteryOptimization', LpMaximize)
charge = LpVariable.dicts('ch', range(n), 0, 1)  # 1 MWh steps
discharge = LpVariable.dicts('dis', range(n), 0, 1)

# Objective
prob += lpSum([(discharge[t] * price[t] * 0.92) - (charge[t]*price[t]/0.92) for t in range(n)])

# SOC constraints
soc = 0
for t in range(n):
    soc += charge[t] - discharge[t]
    prob += soc <= 2
    prob += soc >= 0
    # cycles cap: ≤2 per day handled approximately by limiting energy moved
prob.solve()
print('Monthly profit (Jan) €', sum([(discharge[t].value()*price[t]*0.92) - 
                                     (charge[t].value()*price[t]/0.92) for t in range(n)]))


Scaling the above MILP to the full year may require aggregation or rolling‑horizon techniques to keep the problem tractable.

## Part III – Consumer Reaction to Price <a id='Part-III'></a>

### 3 A. National‑Level Demand Elasticity

In [ ]:

nat_cons_df['timestamp'] = pd.to_datetime(nat_cons_df['timestamp'])
nat_cons_df = nat_cons_df.merge(day_prices_df[['timestamp','price_EUR_MWh']],
                                on='timestamp', how='left')
nat_cons_df = nat_cons_df.merge(meteo_df[['time','avg_temperature','avg_ghi']],
                                left_on=nat_cons_df['timestamp'].dt.floor('H'),
                                right_on='time', how='left').drop(columns=['key_0'])
nat_cons_df['log_demand'] = np.log(nat_cons_df['demand_MWh'])
nat_cons_df['log_price'] = np.log(nat_cons_df['price_EUR_MWh'])

X = sm.add_constant(nat_cons_df[['log_price','avg_temperature']])
y = nat_cons_df['log_demand']
nat_model = sm.OLS(y, X, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags':24})
print(nat_model.summary())

elasticity = nat_model.params['log_price']
print('Estimated price elasticity:', elasticity)


### 3 B. Object‑Level Panel Regression

In [ ]:

# Aggregate hourly price/weather
hour_ref = day_prices_df[['timestamp','price_EUR_MWh']].copy()
hour_ref['timestamp'] = pd.to_datetime(hour_ref['timestamp'])
meteo_df['time'] = pd.to_datetime(meteo_df['time'])
hour_ref = hour_ref.merge(meteo_df, left_on='timestamp', right_on='time', how='left').drop(columns=['time'])

obj_cons_df['consumptionTime'] = pd.to_datetime(obj_cons_df['consumptionTime'])
panel = obj_cons_df.merge(hour_ref, left_on='consumptionTime', right_on='timestamp', how='left')
panel['log_cons'] = np.log(panel['amount']+1e-6)
panel['log_price'] = np.log(panel['price_EUR_MWh'])
panel['hour'] = panel['consumptionTime'].dt.hour
# Fixed‑effects via within transformation using statsmodels
import linearmodels as lm
panel.set_index(['objectNumber','consumptionTime'], inplace=True)

fe_mod = lm.PanelOLS.from_formula('log_cons ~ 1 + log_price + avg_temperature + EntityEffects + TimeEffects',
                                  data=panel)
fe_res = fe_mod.fit(cov_type='clustered', cluster_entity=True)
print(fe_res.summary)


### 3 C. Visualising Behavioural Shifts

In [ ]:

sns.scatterplot(data=panel.sample(10000), x='price_EUR_MWh', y='amount', alpha=0.2)
plt.xscale('log'); plt.yscale('log')
plt.title('Sample Object‑Level Demand vs Price')
plt.xlabel('Price (EUR/MWh)')
plt.ylabel('Consumption (kWh)')
plt.show()


## Conclusions <a id='Conclusions'></a>
*Summarise key insights, profitability ranges, and policy implications here.*